In [1]:
#Categorizar una imagen de internet
from PIL import Image
import requests
from io import BytesIO
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import load_model


modelo = "LSDENSENET100"

custom_objects = {'KerasLayer': hub.KerasLayer}
modelo = tf.keras.models.load_model(f'./models/LS/{modelo}.h5', custom_objects=custom_objects)
hermandadesLS = {0:"LAS AGUAS", 1:"SAN GONZALO", 2:"SAN PABLO", 3:"SANTA GENOVENA", 4:"SANTA MARTA", 5:"VERA CRUZ"}

In [46]:
def categorizar(url, hermandades):
    respuesta = requests.get(url)
    img = Image.open(BytesIO(respuesta.content))
    img = np.array(img).astype(float)/255

    img = cv2.resize(img, (224,224))
    prediccion = modelo.predict(img.reshape(-1, 224, 224, 3))
    print(prediccion[0])
    if max(prediccion[0]) >= 0.7:
        top = np.argmax(prediccion[0], axis=-1)
        return hermandades[top]
    else:
        index = np.argpartition(prediccion[0], -3)[-3:]
        indices = index[np.argsort(-prediccion[0][index])]
        return [hermandades[i] for i in indices]

In [43]:
def categorizar_local(ruta_imagen, hermandades):
    img = cv2.imread(ruta_imagen)
    
    if img is None:
        raise Exception(f"No se pudo cargar la imagen en {ruta_imagen}")
    img = cv2.resize(img, (224, 224))
    img = img / 255.0
    img = img.reshape(-1, 224, 224, 3)
    prediccion = modelo.predict(img)
    print(prediccion[0])
    if max(prediccion[0]) >= 0.7:
        top = np.argmax(prediccion[0], axis=-1)
        return hermandades[top]
    else:
        index = np.argpartition(prediccion[0], -3)[-3:]
        indices = index[np.argsort(-prediccion[0][index])]
        return [hermandades[i] for i in indices]

In [48]:

url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSHZF48xe_ksxgPEFYgK6jn2AcySDWptCPEgw&usqp=CAU'

print(categorizar(url, hermandadesLS))


1/1 [==============================] - 0s 192ms/step
[4.1433513e-02 1.7419460e-04 2.6818213e-01 6.6042215e-01 2.9714208e-02
 7.3881558e-05]
['SANTA GENOVENA', 'SAN PABLO', 'LAS AGUAS']


In [ ]:
ruta_imagen = 'C:\\Users\\super\\OneDrive\\Escritorio\\Captura de pantalla 2024-01-17 184200.png'
print(categorizar_local(ruta_imagen, hermandadesLS))